# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
import sys
key = "C:\\Users\\Kelly\\Desktop\\KEYS"
sys.path.append(key)
from GoogleAPI import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv('clean_city_data.csv')
city_df

,Unnamed: 0,City ID,City Name,Cloudiness,Wind Speed,Country,Temperature,Feels Like,Min Temp,Max Temp,Barometric Pressure,Humidity,Sea Level,Ground Level,Latitude,Longitude
0,0,5106841,Albany,100.0,3.0,US,23.56,16.84,23.00,24.01,1026.0,41.0,NaN,NaN,-85.163514,95.948811
1,1,2027316,Ayan,100.0,306.0,RU,5.13,-24.20,-5.13,-5.13,1010.0,90.0,1010.0,998.0,58.152459,136.429368
2,2,964432,Port Alfred,1.0,7.0,ZA,0.63,64.13,63.00,63.00,1017.0,86.0,NaN,NaN,-83.112210,47.904390
3,3,3466704,Castro,0.0,317.0,BR,62.89,65.77,62.89,62.89,1014.0,93.0,1014.0,903.0,-46.959554,-87.603119
4,4,4030556,Rikitea,0.0,33.0,PF,77.92,81.05,77.92,77.92,1017.0,67.0,1017.0,1014.0,-59.957686,-133.391843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,541,286621,Salalah,0.0,20.0,OM,7.34,68.05,73.40,73.40,1017.0,46.0,NaN,NaN,-19.847552,-47.312803
542,542,2070998,Geraldton,0.0,180.0,AU,0.77,68.99,77.00,77.00,1011.0,44.0,NaN,NaN,52.930079,-168.576467
543,543,1853174,Sakaiminato,75.0,270.0,JP,36.34,21.63,35.60,37.40,1021.0,80.0,NaN,NaN,-20.193303,112.679478
544,544,2873427,Markdorf,100.0,70.0,DE,38.23,33.78,35.60,41.00,1017.0,100.0,NaN,NaN,-38.794236,120.708837


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=gkey)
city_df = city_df.dropna()
locations = city_df[["Latitude", "Longitude"]].astype(float)

weights = city_df["Humidity"].astype(float)
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

In [4]:
fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather = city_df.loc[city_df['Humidity'] < 50]
ideal_weather = city_df.loc[city_df['Humidity'] > 30]
ideal_weather = city_df.loc[city_df['Feels Like'] < 74]
ideal_weather = city_df.loc[city_df['Feels Like'] > 68]
ideal_weather = city_df.loc[city_df['Max Temp'] < 84]
ideal_weather = city_df.loc[city_df['Min Temp'] > 58]
ideal_weather = city_df.loc[city_df['Wind Speed'] < 24]
ideal_weather = city_df.loc[city_df['Cloudiness'] > 40]
dropped_weather = ideal_weather.dropna(subset=['Humidity', 'Feels Like', 'Max Temp', 'Min Temp', 'Wind Speed', 'Cloudiness', 'Latitude', 'Longitude'])
hotel_checker = dropped_weather.sample(10)
hotel_checker

,Unnamed: 0,City ID,City Name,Cloudiness,Wind Speed,Country,Temperature,Feels Like,Min Temp,Max Temp,Barometric Pressure,Humidity,Sea Level,Ground Level,Latitude,Longitude
499,499,2016307,Solnechnyy,100.0,148.0,RU,0.05,-8.00,0.05,0.05,1010.0,83.0,1010.0,972.0,-58.729842,-152.564334
300,300,3698359,Chicama,77.0,189.0,PE,6.98,70.20,69.80,69.80,1014.0,74.0,1014.0,999.0,67.503797,179.820844
66,66,3697838,Contamana,67.0,24.0,PE,76.05,82.51,76.05,76.05,1009.0,84.0,1009.0,990.0,53.042258,79.365879
227,227,4001056,La Orilla,79.0,293.0,MX,74.86,79.59,74.86,74.86,1012.0,85.0,1012.0,1009.0,-69.847087,120.737828
172,172,3380290,Sinnamary,84.0,71.0,GF,77.72,83.95,77.72,77.72,1011.0,86.0,1011.0,1010.0,-58.480371,84.711898
101,101,1214488,Meulaboh,100.0,101.0,ID,77.85,83.52,77.85,77.85,1013.0,79.0,1013.0,1012.0,52.905988,118.518921
526,526,1293960,Taunggyi,99.0,162.0,MM,62.69,64.53,62.69,62.69,1018.0,81.0,1018.0,863.0,-1.962302,-99.823293
152,152,1507390,Dikson,96.0,117.0,RU,15.56,-24.03,-15.56,-15.56,1014.0,89.0,1014.0,1007.0,-1.548659,74.601031
190,190,5961417,Gimli,100.0,160.0,CA,9.16,-4.16,6.80,10.40,1018.0,78.0,1018.0,989.0,-5.340987,-107.168054
283,283,1649150,Bengkulu,100.0,296.0,ID,78.82,83.43,78.82,78.82,1013.0,86.0,1013.0,1010.0,-29.665600,-114.245282


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = hotel_checker.reindex(columns = ["City Name", "Country", "Latitude", "Longitude"])
hotel_df["Hotel Name"] = ""
hotel_df = hotel_df.reset_index().rename(columns={"City Name":"City", "Latitude":"Lat", "Longitude":"Lng"})
hotel_df

,index,City,Country,Lat,Lng,Hotel Name
0,499,Solnechnyy,RU,-58.729842,-152.564334,
1,300,Chicama,PE,67.503797,179.820844,
2,66,Contamana,PE,53.042258,79.365879,
3,227,La Orilla,MX,-69.847087,120.737828,
4,172,Sinnamary,GF,-58.480371,84.711898,
5,101,Meulaboh,ID,52.905988,118.518921,
6,526,Taunggyi,MM,-1.962302,-99.823293,
7,152,Dikson,RU,-1.548659,74.601031,
8,190,Gimli,CA,-5.340987,-107.168054,
9,283,Bengkulu,ID,-29.665600,-114.245282,


In [7]:
# find the closest restaurant of each type to coordinates


placeid_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
coordinate_url = "https://maps.googleapis.com/maps/api/place/details/json?"
hotel_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

hotels = []
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    #get place_id for city name
    placeid_params = {
        "input": row['City'],
        "inputtype": "textquery",
        "key": gkey,
    }
    
    ID = requests.get(placeid_url, params=placeid_params).json()
    ID = (ID["candidates"][0]["place_id"])
    City = row['City']
    print(f'City = {City}')
    

#     use place_id to get coordinates that agree with google's api
    coordinate_params = {
        "place_id": ID,
        "key": gkey,
    }
#     assemble url and make API request
    coords_req = requests.get(coordinate_url, params=coordinate_params).json()
    lat = coords_req['result']['geometry']['location']['lat']
    lng = coords_req['result']['geometry']['location']['lng']
    try:
        Country = row["Country"]
        print(f'Country = {Country}')
    except:
        print(f'Country = n/a')

    
    coords = f'{lat},{lng}'
    
    hotel_params = {
        "location": coords,  
        "radius": 5000,
        "types": "lodging",
        "key": gkey,
    }
    
#     assemble url and make API request
    print(f"Retrieving Results for hotels near {City}:")
    response = requests.get(hotel_url, params=hotel_params).json()
    # extract results
    try:
        results = response['results'][0]['name']
        hotels.append(results)
        print(f'Hotel = {results}')
    except:
        results = "No hotel found nearby"
        hotels.append("n/a")
        print(results)
    
    
    if results == "No hotel found nearby":
        print(f"Sorry, there are no hotels within 5000 meters of {City}. Maybe try someplace less remote?")
    else:
        print(f"The closest hotel to {City} is {results}.")    
    print("------------")
hotel_df["Hotel Name"] = hotels
hotel_df.rename(columns={"Latitude":"Lat", "Longitude":"Lng"})
hotel_df


City = Solnechnyy
Country = RU
Retrieving Results for hotels near Solnechnyy:
Hotel = Hotel complex Pastoral
The closest hotel to Solnechnyy is Hotel complex Pastoral.
------------
City = Chicama
Country = PE
Retrieving Results for hotels near Chicama:
Hotel = Hotel Sueños de Chicama
The closest hotel to Chicama is Hotel Sueños de Chicama.
------------
City = Contamana
Country = PE
Retrieving Results for hotels near Contamana:
Hotel = Fundo Portada del Sol
The closest hotel to Contamana is Fundo Portada del Sol.
------------
City = La Orilla
Country = MX
Retrieving Results for hotels near La Orilla:
Hotel = Meadowmere Resort
The closest hotel to La Orilla is Meadowmere Resort.
------------
City = Sinnamary
Country = GF
Retrieving Results for hotels near Sinnamary:
Hotel = ALETHEIA Appartement Sinnamary
The closest hotel to Sinnamary is ALETHEIA Appartement Sinnamary.
------------
City = Meulaboh
Country = ID
Retrieving Results for hotels near Meulaboh:
Hotel = Kost Pria
The closest hot

,index,City,Country,Lat,Lng,Hotel Name
0,499,Solnechnyy,RU,-58.729842,-152.564334,Hotel complex Pastoral
1,300,Chicama,PE,67.503797,179.820844,Hotel Sueños de Chicama
2,66,Contamana,PE,53.042258,79.365879,Fundo Portada del Sol
3,227,La Orilla,MX,-69.847087,120.737828,Meadowmere Resort
4,172,Sinnamary,GF,-58.480371,84.711898,ALETHEIA Appartement Sinnamary
5,101,Meulaboh,ID,52.905988,118.518921,Kost Pria
6,526,Taunggyi,MM,-1.962302,-99.823293,True Treasure Hotel
7,152,Dikson,RU,-1.548659,74.601031,n/a
8,190,Gimli,CA,-5.340987,-107.168054,Lakeview Gimli Resort
9,283,Bengkulu,ID,-29.665600,-114.245282,Grage Hotel Bengkulu


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:

# Add marker layer ontop of heat map

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…